# Sample System Solver
We demonstrate here a simple example of how the broyden and solver_helper libary files can be used to solve a system of particles in the framework of a RMF mean field model. 

### Relevant Background 
A neutron star core is essentially a "gas" of particles such as protons, neutrons, and electrons. Realistically, these particles interact with one another in a non-trivial way (as opposed to say an ideal gas). We can describe such a system using a Lagrangian-density which consists of a free part which describes the kinetic energy of the constituent particles and an interaction part, the non-linear term that couples the various particles together. 

In the context of a relativistic mean field model such as a Walecka-type model, this interaction Lagrangian is what couples the baryons to mesons. Mesons in this sense act as a sort of mediator between otherwise non-interacting particles. 
$$
  \mathcal{L} = \mathcal{L}_\text{Baryon} + \mathcal{L}_\text{Meson} + \mathcal{L}_\text{Int}
$$
From this Lagrangian, we get the equations of motion for the meson fields by applying the Euler-Lagrange equations of motion. We apply the mean field approximation which simplifies the equations of motion (the time-dependent terms vanish and the spatial gradient terms vanish as well). The interpretation is that we treat baryons fully, and baryons move and interact against a static meson background. In the case below, we have the following equations for the field values for the $\sigma, \omega, \rho,\phi$ mesons
$$
    m^2_\sigma \sigma + \frac{dU}{d\sigma} = \sum_i n_i^s\\
    m^2_\omega \omega = \sum_i g_{\omega i}n_i\\
    m^2_\rho \rho = \sum_i g_{\rho i} n_i\\
    m^2_\phi \phi = \sum_i g_{\phi i}n_i
$$
where the sums over $i$ sum over the baryons. The $g_i$ terms are coupling constants that couple the corresponding mesons to their corresponding baryon. Finally, we have
$$
    n_i^s = \frac{m_i^*}{2\pi}\left[E_{F_i}^* k_{F_i} - {m_i^*}^2 \ln \frac{k_{F_i} + E_{F_i}^*}{m_i^*} \right] \qquad n_i = \frac{k_{F_i}^3}{3\pi^2}
$$

Finally to add to these equations of motion, we have three additional conditions

<b> Charge neutrality. </b>

The number of negative and positive particles must be equal. 
    $$
        \sum_i n_- + \sum_i n_+ = 0 \qquad n_p + n_e = 0
    $$

<b> Baryon number conservation. </b>

The number density of baryons must be equal to the sum of the number densities of all other baryons. 
    $$
        n_B = \sum_B n_B \qquad n_B = n_n + n_p
    $$
<b> Beta equilibrium. </b> 

For each baryon, there is a constraint on the chemical potentials.
    $$
        \mu_B = \mu_n - q_B \mu_e \qquad \mu_n = \mu_p + \mu_e
    $$


### Usage of the Code 
1. We start by declaring an equation of state (EOS) object by calling eos() and providing as argument the relevant coupling constants. In our case, we provide coupling constants for nucleons and separate ones for hyperons. 

2. Then we create our system by making lists for: baryons, mesons, and leptons. The baryons, mesons, and leptons are pre-stored/defined with all of their relevant masses, charges, etc in the solver_helper file. 

3. Then we call init(), a function that fills in the baryon lists with the values of the coupling constant. We could re-write this so it gets filled into the full solver function in the future to be cleaner. 

4. Finally, we call full_solve, which takes the aforementioned system and generates values for the meson fields, baryon and lepton momenta and fractions and returns a Pandas DataFrame with the corresponding values. This DataFrame can then be exported to a csv file really easily. 

In [1]:
from broyden import *
from solver_helper import *

#### Setting up the System 

In [5]:
# gm3 model 
gm3 = eos(g_sigma_N = 8.784820, g_omega_N = 8.720086, g_rho_N = 8.544795, b = 0.008628, c = -0.002433,\
         g_sigma_H = 5.408849, g_omega_H = 5.813391, g_rho_H = 0.0, g_phi_H = -4.110688)
n0 = gm3.n0 

# NPE 
baryon_list = [Neutron, Proton]
meson_list = [sigma, omega, rho, phi]
lepton_list = [electron]

init(gm3, baryon_list, meson_list)

#### Calling full_solve to generate the data
1. This takes about 10-15 minutes on average to fully solve. Would be much faster in C/C++ probably.

In [6]:
data = full_solve(gm3, baryon_list, lepton_list, meson_list, [7.94, 4.51, -2.24, 0.0, 210.426, 43.23, 43.23])

In [7]:
data

,nB/n0,sigma field (MeV),rho field (MeV),omega field (MeV),phi field (MeV),Neutron kF (MeV),Proton kF (MeV),electron kF (MeV),Neutron frac,Proton frac,electron frac
0,0.27,7.941193,-2.247962,4.514730,1.326330e-29,210.426938,43.238314,43.238314,0.991399,0.008601,0.008601
1,0.28,8.199099,-2.329050,4.681942,-1.235150e-27,212.960614,44.528049,44.528049,0.990942,0.009058,0.009058
2,0.29,8.455030,-2.409949,4.849154,-4.714163e-28,215.432600,45.809033,45.809033,0.990477,0.009523,0.009523
3,0.30,8.709030,-2.490655,5.016367,-4.098196e-28,217.846350,47.081534,47.081534,0.990006,0.009994,0.009994
4,0.31,8.961139,-2.571167,5.183579,-4.761129e-28,220.205018,48.345801,48.345801,0.989528,0.010472,0.010472
...,...,...,...,...,...,...,...,...,...,...,...
768,7.95,75.202688,-31.660986,132.933718,-2.541296e-27,588.110921,418.542097,418.542097,0.735053,0.264947,0.264947
769,7.96,75.229647,-31.685294,133.100930,-2.572371e-27,588.326705,418.778116,418.778116,0.734938,0.265062,0.265062
770,7.97,75.256548,-31.709590,133.268142,-3.760202e-28,588.542307,419.013916,419.013916,0.734823,0.265177,0.265177
771,7.98,75.283392,-31.733873,133.435355,-1.138516e-27,588.757728,419.249498,419.249498,0.734709,0.265291,0.265291


#### Saving data to csv file 

In [ ]:
data.to_csv('npe_data.csv')